In [1]:
from test_utils import *
tag = f"c0"
weight_dict = load_weight_dict_by_tag('../out/FedTest5-resnet18/2024-10-19-11:47:36/grad_lists', tag)
# cal_similay_by_tag('weight_lists/202405281805-avg-ss/',"server", unselect_keys=['bn', 'num_batches_tracked','downsample'], describe='202405281805-avg-ss')


/home/zhengsl/code/FL-bench/test_similar/test_utils.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight = torch.load(weight_path)


In [6]:
filter = LayerFilter(unselect_keys=['num_batches_tracked','running_mean','running_var'], all_select_keys=['layer'])

In [9]:
for i in filter(weight_dict[0][0]).keys():
    print(f"'{i}' : {weight_dict[0][0][i].shape},")

'base.layer1.0.conv1.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.0.bn1.weight' : torch.Size([64]),
'base.layer1.0.bn1.bias' : torch.Size([64]),
'base.layer1.0.conv2.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.0.bn2.weight' : torch.Size([64]),
'base.layer1.0.bn2.bias' : torch.Size([64]),
'base.layer1.1.conv1.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.1.bn1.weight' : torch.Size([64]),
'base.layer1.1.bn1.bias' : torch.Size([64]),
'base.layer1.1.conv2.weight' : torch.Size([64, 64, 3, 3]),
'base.layer1.1.bn2.weight' : torch.Size([64]),
'base.layer1.1.bn2.bias' : torch.Size([64]),
'base.layer2.0.conv1.weight' : torch.Size([128, 64, 3, 3]),
'base.layer2.0.bn1.weight' : torch.Size([128]),
'base.layer2.0.bn1.bias' : torch.Size([128]),
'base.layer2.0.conv2.weight' : torch.Size([128, 128, 3, 3]),
'base.layer2.0.bn2.weight' : torch.Size([128]),
'base.layer2.0.bn2.bias' : torch.Size([128]),
'base.layer2.0.downsample.0.weight' : torch.Size([128, 64, 1, 1]),
'base.layer2.0.downs

In [18]:
weight_0 = filter(weight_dict[0][0])
total_size = calculate_data_size(weight_0)
for i in weight_0.keys():
    size = cal_memory(weight_0[i])
    ratio = size / total_size
    print(f"{i:35} | {size:7} bytes | {ratio * 100:.2f}% | shape: {weight_0[i].shape}")

base.layer1.0.conv1.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.0.bn1.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.bn1.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.conv2.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.0.bn2.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.0.bn2.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.conv1.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.1.bn1.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.bn1.bias              |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.conv2.weight          |  147456 bytes | 0.33% | shape: torch.Size([64, 64, 3, 3])
base.layer1.1.bn2.weight            |     256 bytes | 0.00% | shape: torch.Size([64])
base.layer1.1.

In [ ]:
from functools import reduce


def cal_size(M, K, D, L):
    if isinstance(M, list):
        M = reduce(lambda x, y: x * y, M)
    w1 = K * M / L 
    w2 = D * L
    w = w1 + w2
    r =  w / M
    print(f"M {M}, size: {w}, ratio: {r}, w1: {w1}, w2: {w2}")
cal_size()

In [1]:
def cal_K_D_L(M, K=None, D=None, L=None):
    # vector = vector.flatten()
    # M = vector.shape[0]
    if K is None and D is not None and L is not None:
        max = L // 2
        K = D * L * L // M
        if K > max:
            raise ValueError(f"Not good. K is too large, max is {max}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is None and L is not None:
        if L // K <= 2:
            raise ValueError(f"Not good. L // K is too small, min is 2, now is {L // K}")
        max = M // (4 * K)
        D = K * M // (L * L)
        if D > max:
            raise ValueError(f"Not good. D is too large, max is {max}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is not None and L is None:
        if K * D * 4 >= M:
            raise ValueError(f"Not good. K * D * 4 is too large, max is {M // 4}, now is {K * D * 4}")
        min = 2 * K
        L = int(math.sqrt(M * K // D))
        if L < min:
            raise ValueError(f"Not good. L is too small, min is {min}")
        else:
            r = 2 * L * D / M
    elif K is not None and D is not None and L is not None:
        if K * D * 4 >= M:
            raise ValueError(f"Not good. K * D * 4 is too large, max is {M // 4}, now is {K * D * 4}")
        if L // K < 2:
            raise ValueError(f"Not good. L // K is too small, min is 2, now is {L // K}")
        r = K / L + D * L / M
    else:
        raise ValueError(f"K, D, L must have one None")
    
    if K < D:
        raise ValueError(f"Not good. K < D, now is {K} < {D}")
    return K, D, L, 1 - r



In [3]:
K, max_D, L, r = cal_K_D_L(512*512*3*3,K=32, D=32, L=256*3)
print(f"K={K}, D={max_D}, L={L}, r={r}")

K=32, D=32, L=768, r=0.9479166666666666


In [302]:
# 遍历K和D，找到最好的K和D，使得r最大
max_r = 0
max_K = 0
max_D = 0
max_L = 0
for K in [2,4,6,8,10,12,14,16,18,20]:
    for max_D in range(1, K):
        for L in range(2*K, 1024):
            try:
                K, max_D, L, r = cal_K_D_L(4096, K=K, D=max_D, L=L)
                if L < 1024 and r > max_r:
                    max_r = r
                    max_K = K
                    max_D = max_D
                    max_L = L

            except:
                continue
    print(f"{K}-th max_K={max_K}, max_D={max_D}, max_L={max_L}, max_r={max_r}")
print(f"max_K={max_K}, max_D={max_D}, max_L={max_L}, max_r={max_r}")


2-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
4-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
6-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
8-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
10-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
12-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
14-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
16-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
18-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
20-th max_K=2, max_D=1, max_L=91, max_r=0.955805181146978
max_K=2, max_D=1, max_L=91, max_r=0.955805181146978


In [19]:
from typing import List, Tuple, Union
import numpy as np
import torch
from torch import Tensor
from typing import Dict
from scipy.sparse.linalg import svds

from src.utils.my_utils import cal_memory


class Compressor:
    # 基类，用于压缩和解压缩
    def __init__(self, **kwargs):
        pass

    def compress(self, tensor:torch.Tensor):
        pass

    def uncompress(self, tensor:torch.Tensor):
        pass
    
    def update_basis(self, update_dict:Dict[int, torch.Tensor]):
        pass

    def update_basis_by_vector(self, vector:torch.Tensor):
        pass

class TopkCompressor(Compressor):
    '''
    TopkCompressor类, 用于基于top-k百分比的有损压缩。
    在压缩时，仅保留绝对值最大的K%个元素，并将结果转换为指定的稀疏矩阵格式；
    在解压缩时，将稀疏矩阵转换回密集矩阵。
    '''

    def __init__(self, k_percent: float, sparse_format: str = 'csr', **kwargs):
        super().__init__(**kwargs)
        self.k_percent = k_percent  # 要保留的最大元素的百分比 (0 < k_percent <= 100)
        if sparse_format.lower() not in ['coo', 'csr']:
            raise ValueError("sparse_format must be one of 'coo' or 'csr'")
        self.sparse_format = sparse_format.lower()

    def compress(self, tensor: torch.Tensor) -> Tuple[Union[torch.Tensor, torch.sparse.Tensor], torch.Tensor]:
        '''
        Args:
            tensor: torch.Tensor, 待压缩的张量
        Returns:
            compressed_tensor: torch.sparse.Tensor, 压缩后的稀疏张量
            error: torch.Tensor, 误差张量
        '''
        # 获取张量的形状和元素总数
        original_shape = tensor.shape
        num_elements = tensor.numel()
        
        # 计算要保留的元素数量（基于百分比）
        k = int(num_elements * (self.k_percent / 100.0))
        
        # 将tensor展平，以便找到全局的k个最大值
        flat_tensor = tensor.flatten()
        abs_tensor = torch.abs(flat_tensor)

        _, indices = torch.topk(abs_tensor, k)
        compressed_tensor = torch.zeros_like(flat_tensor)
        compressed_tensor[indices] = flat_tensor[indices]
        compressed_tensor = compressed_tensor.unsqueeze(0)

        # 将压缩后的张量转换为稀疏张量
        if self.sparse_format == 'coo':
            compressed_sparse_tensor = compressed_tensor.to_sparse_coo()
        elif self.sparse_format == 'csr':
            compressed_sparse_tensor = compressed_tensor.to_sparse_csr()
        # 计算误差
        error = tensor - compressed_tensor.reshape(original_shape)
        return compressed_sparse_tensor, error

    def uncompress(self, sparse_tensor: torch.sparse.Tensor, shape: Tuple[int, ...] = None) -> torch.Tensor:
        '''
        Args:
            sparse_tensor: torch.sparse.Tensor, 压缩后的稀疏张量
            shape: tuple, 原始张量的形状（如果需要的话）
        Returns:
            torch.Tensor, 解压后的密集张量
        '''
        # 将稀疏张量转换回密集张量
        if sparse_tensor.layout == torch.sparse_coo or sparse_tensor.layout == torch.sparse_csr:
            dense_tensor = sparse_tensor.to_dense()
            dense_tensor = dense_tensor.squeeze(0)
        
        # 如果提供了形状，重新塑形
        if shape is not None:
            dense_tensor = dense_tensor.reshape(shape)
        
        return dense_tensor

    def update_basis(self, update_dict: dict):
        # 对于top-k压缩，无需更新基
        pass

    def update_basis_by_vector(self, vector: torch.Tensor):
        # 对于top-k压缩，无需通过向量更新基
        return {}

class SVDCompressor(Compressor):
    '''
    SVDCompress类, 用于SVD压缩和解压缩, 用于SVDFed算法
    '''
    def __init__(self, L, R, use_scale=True, **kwargs):
        self.U:torch.Tensor = None
        self.L = L # 调整alpha
        self.R = R # 误差阈值
        self.use_scale = use_scale


    def update_basis(self, update_dict:Dict[int, torch.Tensor]):
        max_index = max(update_dict.keys())
        key = next(iter(update_dict))
        L = update_dict.get(key).shape[0]
        self.U = torch.zeros(L, max_index+1)
                    
        # key是更新位置，value是更新的值
        for k, v in update_dict.items():
            self.U[:,k] = v.clone().detach()

    def get_svd_error(self, vector_t, U):
        # 依次计算grads中每个梯度的误差
        total_error = 0
        for i in range(vector_t.shape[1]):
            g = vector_t[:,i].squeeze()
            alpha = U.T @ g
            # tmp = U @ alpha
            # alpha = min(g.norm() / tmp.norm(), self.L) * alpha
            g_approx = U @ alpha
            error = g - g_approx
            total_error += error.norm()/g.norm()
        return total_error/vector_t.shape[1]
    
    def update_basis_by_vector(self, vector:torch.Tensor):
        '''
        Return update_dict
        '''
        update_dict = {}
        vector_t = vector.T
        U, S, _ = torch.linalg.svd(vector_t, full_matrices=False)

        k = 0
        l = 0
        r = len(S)
        while l < r:
            mid = (l + r) // 2
            t = self.get_svd_error(vector_t, U[:,:mid])
            if t < self.R:
                r = mid
            else:
                l = mid + 1
        k = l

        self.U = U[:,:k]
        print(f"Layer basis selected maximun: {k}, error: {t}, threshold: {self.R}")
        # update_dict 为全部U向量
        for i in range(k):
            update_dict[i] = self.U[:,i]
        return update_dict
    
    def compress(self, vector:torch.Tensor):
        '''
        Args:
            vector: torch.Tensor, 压缩的张量
        Returns:
            a: torch.Tensor, 张量在基下的投影
            e: torch.Tensor, 压缩张量的误差
        '''
        vector_t = vector.flatten().unsqueeze(0).T
        if self.U is None:
            return vector, torch.zeros_like(vector)
        alpha = self.U.T @ vector_t
        _g = self.U @ alpha

        if self.use_scale:
            scale = min((torch.std(vector_t)  + 1e-8)/ (torch.std(_g)  + 1e-8), self.L)
        else:
            scale = 1

        alpha = scale*alpha
        e = vector_t - scale*_g
        return alpha, e.T.reshape(vector.shape)
    
    def uncompress(self, alpha:torch.Tensor, shape = None):
        '''
        Args:
            alpha: torch.Tensor, 压缩后的alpha
            shape: tuple, 原始张量的shape
        Returns:
            torch.Tensor, 解压后的张量
        '''
        # 如果a的维度刚好等于shape，直接返回
        if alpha.shape == shape:
            return alpha.clone().detach()
        elif shape is None:
            return (self.U @ alpha).T
        else:
            return (self.U @ alpha).T.reshape(shape)


class QuickSlideSVDCompressor(Compressor):
    def __init__(self, K, max_D, L, u_dtype='float32',**kwargs):
        self.K = K # U的列数
        self.max_D = max_D # 主动更新的维度
        self.L = L # 参数切片的长度
        self.U = None # 基础的U

        self.cur_D = max_D
        if u_dtype == 'float128':
            self.dtype = np.float128
        elif u_dtype == 'float64':
            self.dtype = np.float64
        elif u_dtype== 'float32':
            self.dtype = np.float32
        elif u_dtype == 'float16':
            self.dtype = np.float16
        else:
            raise ValueError(f"Unsupported dtype {u_dtype}")
        # self.dtype = np.float16

    def update_basis(self, update_dict: dict):
        if self.U is None:
            assert len(update_dict) == self.K, f"First update_dict length must be {self.K}"
            max_index = max(update_dict.keys())
            self.U = np.zeros((self.L, max_index + 1), dtype=self.dtype)  # Initialize U as numpy array
        
        # Key is the update index, value is the update value
        for k, v in update_dict.items():
            self.U[:, k] = v.copy()

    def update_basis_by_vector(self, vector:torch.Tensor, update_threshold:float=0):
        '''
        Return update_dict
        '''
        # 通过向量更新U
        flatten_L = vector.numel()
        if flatten_L % self.L != 0:
            return {}
        vector = vector.reshape(-1, self.L)
        if self.K > vector.shape[0]:
            raise ValueError(f"K {self.K} must less than vector.shape[0] {vector.shape[0]}")
        update_dict = {}
        vector_t = vector.T.cpu().numpy()
        if self.U is None:
            # Compute U using SVD decomposition
            U, S, V = svds(vector_t, k=self.K)
            self.U = U.copy().astype(self.dtype)
            # All U vectors are updated
            for i in range(self.K):
                update_dict[i] = self.U[:, i].copy()
        
        elif self.cur_D > 0:
            # Reconstruct vector using U
            e = vector_t - self.U @ self.U.T @ vector_t
            U_e, S_e, V_e = svds(e, k=self.cur_D)
            U_K_e = np.hstack([self.U, U_e], dtype=self.dtype)  # Concatenate the new basis vectors

            alpha = U_K_e.T @ vector_t
            
            contribution = np.sum(alpha ** 2, axis=1)  # Calculate the contribution of each orthogonal vector
            min_indices = np.argsort(contribution)[:self.cur_D]  # Get the D smallest contributing indices

            min_indices_set = set(min_indices.tolist())
            wait_D_update_set = set(range(self.K, self.K + self.cur_D))
            sub_index = min_indices_set - wait_D_update_set
            add_index = wait_D_update_set - min_indices_set

            # 
            # Swap columns
            U_K_e[:, list(sub_index)] = U_K_e[:, list(add_index)]
            alpha[list(sub_index)] = alpha[list(add_index)]
            U_K = U_K_e[:, :self.K]
            if update_threshold > 0:
                alpha_2 = alpha[:self.K]
                e_2 = vector_t - U_K @ alpha_2
                # Check if error difference is below the threshold, and skip update if so
                if (np.linalg.norm(e) - np.linalg.norm(e_2)) / np.linalg.norm(e) < update_threshold:
                    return {}
            
            self.U = U_K.copy()
            # Return updated columns in dictionary form
            for i in sub_index:
                update_dict[i] = U_K_e[:, i].copy()
            
            # 动态调整更新的维度
            actual_D = len(sub_index)   # 实际更新的维度
            if actual_D >= self.cur_D//2:
                self.cur_D = min(self.max_D, self.cur_D * 2)
            
            elif self.cur_D//4 < actual_D < self.cur_D//2:
                self.cur_D = max(2, self.cur_D * 3 // 4)
            else:
                self.cur_D = max(2, self.cur_D // 2)
            print(f"actual_D/cur_D: {actual_D}/{self.cur_D}")
        return update_dict
    
    def compress(self, vector:torch.Tensor):
        '''
        Args:
            vector: torch.Tensor, 压缩的张量, 若vector的最后一个维度不能被L整除, 则返回自身, 否则返回压缩后的张量
        Returns:
            a: torch.Tensor, 张量在基下的投影
            e: torch.Tensor, 压缩张量的误差
        '''
        flatten_L = vector.numel()
        if flatten_L % self.L != 0:
            print(f"vector.numel() {flatten_L} can't divide L {self.L}. Return itself")
            return vector, torch.zeros_like(vector)
   
        vector_t = vector.reshape(-1, self.L).T
        # 通过U重构vector
        U = torch.from_numpy(self.U).to(vector.device, dtype=vector.dtype)
        alpha = U.T @ vector_t
        g = U @ alpha
        e = vector_t - g
        return alpha, e.T.reshape(vector.shape)

    def uncompress(self, alpha:torch.Tensor, shape = None):
        # 如果a的维度刚好等于shape，直接返回
        if alpha.shape == shape:
            return alpha.clone().detach()
        elif shape is None:
            U = torch.from_numpy(self.U).to(alpha.device, dtype=alpha.dtype)
            return (U @ alpha).T
        else:
            U = torch.from_numpy(self.U).to(alpha.device, dtype=alpha.dtype)
            return (U @ alpha).T.reshape(shape)


class SlideSVDCompressor(Compressor):
    def __init__(self, K, D, L, device='cpu'):
        self.K = K # U的列数
        self.D = D # 主动更新的维度
        self.L = L # 参数切片的长度
        self.U = None # 基础的U
        self.device = device

    def update_basis(self, update_dict:Dict[int, torch.Tensor]):
        if self.U is None:
            assert len(update_dict) == self.K, f"First update_dict length must be {self.K}"
            max_index = max(update_dict.keys())
            self.U = torch.zeros(self.L, max_index+1, device=self.device)
        
        # key是更新位置，value是更新的值
        for k, v in update_dict.items():
            self.U[:,k] = v.clone().detach().to(self.device)

    def update_basis_by_vector(self, vector:torch.Tensor, update_threshold:float=0):
        '''
        Return update_dict
        '''
        # 通过向量更新U
        flatten_L = vector.numel() if len(vector.shape) == 1 else (vector.numel() // vector.shape[0])
        if flatten_L % self.L != 0:
            return {}
        vector = vector.reshape(-1, self.L)
        if self.K > vector.shape[0]:
            raise ValueError(f"K {self.K} must less than vector.shape[0] {vector.shape[0]}")
        update_dict = {}
        vector_t = vector.T
        if self.U is None:
            # 通过SVD分解得到U
            U, S, V = torch.linalg.svd(vector_t, full_matrices=False)
            self.U = U[:,:self.K].to(self.device)
            # update_dict 为全部U向量
            for i in range(self.K):
                update_dict[i] = self.U[:,i]
        
        elif self.D > 0:
            # 通过U重构vector
            e = vector_t - self.U @ self.U.T @ vector_t
            U_e, S_e, V_e = torch.linalg.svd(e, full_matrices=False)
            U_K_e = torch.cat([self.U, U_e[:,:self.D]], dim=1)

            alpha = U_K_e.T @ vector_t
            
            contribution = torch.sum(alpha ** 2, dim=1)  # 计算每个正交向量的贡献度（平方和）
            _, min_indices = torch.topk(contribution, k=self.D, largest=False)

            min_indices_set = set(min_indices.tolist())
            wait_D_update_set = set([i for i in range(self.K, self.K+self.D)])
            sub_index = min_indices_set - wait_D_update_set
            add_index = wait_D_update_set - min_indices_set

            # 交换列
            U_K_e[:,list(sub_index)] = U_K_e[:,list(add_index)]
            alpha[list(sub_index)] = alpha[list(add_index)]
            U_K = U_K_e[:,:self.K]
            alpha_2 = alpha[:self.K]
            # alpha_2 = U_K.T @ vector_t
            # alpha_2不需要重新计算，通过alpha和U_K_e的更新列计算
            e_2 = vector_t - U_K @ alpha_2

            # 若更新后的误差变化小于阈值，则不更新
            # print(f"de {(e.norm() - e_2.norm())/e.norm()}, update_threshold {update_threshold}")
            if (e.norm() - e_2.norm())/e.norm() < update_threshold:
                return {}
            
            self.U = U_K_e[:,:self.K]
            # 返回更新列字典
            for i in sub_index:
                update_dict[i] = U_K_e[:,i].clone().detach()
        
        return update_dict

    def compress(self, vector:torch.Tensor):
        '''
        Args:
            vector: torch.Tensor, 压缩的张量, 若vector的最后一个维度不能被L整除, 则返回自身, 否则返回压缩后的张量
        Returns:
            a: torch.Tensor, 张量在基下的投影
            e: torch.Tensor, 压缩张量的误差
        '''
        flatten_L = vector.numel() if len(vector.shape) == 1 else (vector.numel() // vector.shape[0])
        if flatten_L % self.L != 0:
            print(f"vector.numel() // vector.shape[0] {flatten_L} can't divide L {self.L}. Return itself")
            return vector, torch.zeros_like(vector)
   
        vector_t = vector.reshape(-1, self.L).T
        # 通过U重构vector
        alpha = self.U.T @ vector_t
        g = self.U @ alpha
        e = vector_t - g
        return alpha, e.T.reshape(vector.shape)

    def uncompress(self, alpha:torch.Tensor, shape = None):
        # 如果a的维度刚好等于shape，直接返回
        if alpha.shape == shape:
            return alpha.clone().detach()
        elif shape is None:
            return (self.U @ alpha).T
        else:
            return (self.U @ alpha).T.reshape(shape)

class CompressorCombin:
    def __init__(self, setting_dict:Dict[str, tuple], class_name='SlideSVDCompressor', device='cpu', **kwargs):
        '''
        CompressorCombin类, 用于组合多个Compress类, 为多层参数提供压缩和解压缩功能
        Args:
            setting_dict: key是参数名, value是元组(K, D, L), K是U的列数, D是主动更新的维度, L是参数切片的长度
        '''
        if not isinstance(setting_dict, dict):
            raise ValueError("setting_dict must be a dict")
        
        compressor = globals()[class_name]

        self.setting_dict = setting_dict
        self.compressor_dict:Dict[str, Compressor] = {}
        for key, value in setting_dict.items():
            if isinstance(value, list):
                value = tuple(value)
            self.compressor_dict[key] = compressor(*value, **kwargs, device=device)
        self.device = device

    def update_basis_by_vector(self, model_params:Dict[str, Tensor]):
        '''
        通过model_params更新全部compressor的基
        Args:
            model_params: 模型参数字典
        Returns:
            dict: 更新字典
        '''
        res = {}
        for key, value in model_params.items():
            if key not in self.compressor_dict:
                continue
            compressor = self.compressor_dict[key]
            res[key] = compressor.update_basis_by_vector(value)
        return res

    def compress(self, model_params:Dict[str, Tensor], can_update_basis_func=None, **kwargs) -> Tuple[Dict[str, Tensor], dict, Dict[str, Tensor]]:
        '''
        压缩combine中全部compressor的参数
        Args:
            model_params: 模型参数字典, 如果key不在compressor_dict中, 则不压缩
            can_update_basis_func: 是否可以更新基函数的函数, 返回True或False
        Returns:
            combin_alpha: 压缩后的alpha字典 
            combin_update_dict: 更新字典
        '''
        combin_alpha = {}
        combin_update_dict = {}
        combin_error = {}
        for key, value in model_params.items():
            if key not in self.compressor_dict:
                combin_alpha[key] = value
                combin_update_dict[key] = {}
                combin_error[key] = torch.zeros_like(value)
                continue
            compressor = self.compressor_dict[key]
            if can_update_basis_func is not None:
                if can_update_basis_func(**kwargs):
                    combin_update_dict[key] = compressor.update_basis_by_vector(value)
                else:
                    combin_update_dict[key] = {}
            combin_alpha[key], combin_error[key] = compressor.compress(value)
            # res = compressor.uncompress(combin_alpha[key], value.shape)
            # sim = cos_similar(value, res)
            # print(f"key {key}, cos_similar {sim}")
        return combin_alpha, combin_update_dict, combin_error

    def uncompress(self, combin_alpha:Dict[str, Tensor], templete_model_params:Dict[str, Tensor]) -> Dict[str, Tensor]:
        '''
        根据combin_alpha解压, 如果key不在compressor_dict中, 则无需解压
        Args:
            combin_alpha: 压缩后的alpha字典
            templete_model_params: 参数模板，用于指定解压后的参数形状
        Returns:
            dict: 解压后的模型参数
        '''
        res = {}
        for key, value in combin_alpha.items():
            if key not in self.compressor_dict:
                res[key] = value
            else:
                res[key] = self.compressor_dict[key].uncompress(value, templete_model_params[key].shape)
        return res
    
    def update(self, combin_update_dict:Dict[str, Dict[int, Tensor]]):
        '''
        更新combine中全部compressor的基
        Args:
            combin_update_dict: key是参数名, value是更新字典, 更新字典的key是更新位置, value是更新的值
        '''
        for key, value in combin_update_dict.items():
            if key not in self.compressor_dict:
                continue
            compressor = self.compressor_dict[key]
            compressor.update_basis(value)

class QSGDQuantizer:
    def __init__(self, num_levels=256):
        self.num_levels = num_levels

    def quantize(self, tensor:torch.Tensor):
        norm = tensor.norm(p=2)
        scale = norm / self.num_levels
        sign = tensor.sign()
        abs_tensor = tensor.abs()
        q = (abs_tensor / scale).floor()
        prob = (abs_tensor / scale) - q
        rand_tensor = torch.rand_like(prob)
        q += torch.where(rand_tensor < prob, torch.ones_like(q), torch.zeros_like(q))
        quantized_tensor = sign * q
        return quantized_tensor, 0, scale
    
    def dequantize(self, quantized_tensor, min_val, scale):
        dequantized_tensor = quantized_tensor * scale
        return dequantized_tensor


In [16]:
# from src.utils.compressor_utils import QuickSlideSVDCompressor
# 512, 256, 3, 3
compresser = QuickSlideSVDCompressor(100, 100, 512, 'float32')
layername = 'base.layer3.0.conv1.weight'
v_5 = weight_dict[0][0][layername]
update_dict = compresser.update_basis_by_vector(v_5)

In [17]:
for i in range(1, 50):
    v_test = weight_dict[i][0][layername]
    a_test, e_test = compresser.compress(v_test)
    g_test = compresser.uncompress(a_test, v_test.shape)
    before = cos_similar(g_test, v_test)

    if i % 1 == 0:
        update_dict = compresser.update_basis_by_vector(v_test, update_threshold=0)
        a_test, e_test_after = compresser.compress(v_test)
        g_test = compresser.uncompress(a_test, v_test.shape)
        after = cos_similar(g_test, v_test)
        if after < before:
            if e_test_after.norm() > e_test.norm():
                print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()} e:{e_test.norm()} after e:{e_test_after.norm()}")
            else:
                print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()} e_test_after < e_test but after_similar < before_similar")
        else:
            print(f"Th-{i}: before:{before:10.5f}, after:{after:10.5f} ==> Update_dict:{update_dict.keys()}")
    else:
        print(f"Th-{i}: before:{before:10.5f} e:{e_test.norm()} after e:{e_test_after.norm()}")

actual_D/cur_D: 76/100
Th-1: before:   0.50335, after:   0.77459 ==> Update_dict:dict_keys([0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 82, 85])
actual_D/cur_D: 62/100
Th-2: before:   0.54977, after:   0.78478 ==> Update_dict:dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 46, 47, 50, 53, 57, 65, 74, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 91, 98])
actual_D/cur_D: 54/100
Th-3: before:   0.59461, after:   0.77198 ==> Update_dict:dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 42, 43, 45, 48, 49, 51, 52, 54, 55, 56, 58, 59,

In [95]:
for i in range(1, 50):
    v_test = weight_dict[i][0]['classifier.fc1.weight']
    a_test, e_test = compresser.compress(v_test)
    g_test = compresser.uncompress(a_test, v_test.shape)
    before = cos_similar(g_test, v_test)
    print(f"{i}-th {before}")

1-th 0.3954788148403168
2-th 0.38685572147369385
3-th 0.38566067814826965
4-th 0.4078787863254547
5-th 0.39017173647880554
6-th 0.38016679883003235
7-th 0.3959686756134033
8-th 0.398908793926239
9-th 0.3915354311466217
10-th 0.4029323160648346
11-th 0.40529799461364746
12-th 0.392452597618103
13-th 0.3967442512512207
14-th 0.3904167711734772
15-th 0.3983614444732666
16-th 0.3839189410209656
17-th 0.39532017707824707
18-th 0.3997047245502472
19-th 0.3955429792404175
20-th 0.3920809030532837
21-th 0.38374629616737366
22-th 0.391897976398468
23-th 0.38946467638015747
24-th 0.3910553753376007
25-th 0.3862779438495636
26-th 0.385198712348938
27-th 0.38837939500808716
28-th 0.3965727388858795
29-th 0.3846668004989624
30-th 0.38984793424606323
31-th 0.3796626925468445
32-th 0.38867342472076416
33-th 0.3884607255458832
34-th 0.3864929974079132
35-th 0.38782989978790283
36-th 0.39541861414909363
37-th 0.3933473825454712
38-th 0.38715362548828125
39-th 0.3878777325153351
40-th 0.3889986276626587